In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "..")

import trax
trax.fastmath.backend_name()
from src.data import data_tools
from pathlib import Path

2022-02-15 15:06:42.889492: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [2]:
import tensorflow as tf
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True, cache_dir="../data/raw")


In [3]:
from src.data import data_tools

image_folder = Path("../data/raw/datasets/flower_photos")
datagen = data_tools.data_generator(
    batch_size = 8, 
    path = image_folder, 
    image_size = (150, 150),
    channels = 3,
    shuffle = True
)

In [4]:
%timeit next(datagen)

2022-02-15 15:09:19.208025: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-15 15:09:19.208099: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


30.8 ms ± 4.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
X, y = next(datagen)
X.shape, y.shape

((8, 150, 150, 3), (8, 150, 150, 3))

In [55]:
from trax import layers as tl
model = tl.Serial(
      tl.ToFloat(),

      tl.Conv(32, (3, 3), (1, 1), 'SAME'),
      tl.LayerNorm(),
      tl.Relu(),
      tl.MaxPool(),

      tl.Conv(64, (3, 3), (1, 1), 'SAME'),
      tl.LayerNorm(),
      tl.Relu(),
      tl.MaxPool(),

      tl.Flatten(),
      tl.Dense(5),
      tl.Softmax()
)

In [56]:
from trax.supervised import training

train_task = training.TrainTask(
    labeled_data=datagen,
    loss_layer=tl.CategoryCrossEntropy(),
    optimizer=trax.optimizers.Adam(0.001),
    n_steps_per_checkpoint=100,
)

# eval_task = training.EvalTask(
#     labeled_data=eval_batches_stream,
#     metrics=[tl.CategoryCrossEntropy(), tl.CategoryAccuracy()],
#     n_eval_batches=20,
# )

In [57]:
training_loop = training.Loop(model, 
                              train_task, 
                              output_dir='./cnn_model2')

training_loop.run(5)

In [58]:
X, y = next(datagen)

In [59]:
yhat = model.forward(X)
yhat.shape

(8, 5)

In [60]:
import trax.fastmath.numpy as np
np.argmax(yhat, axis=1)

DeviceArray([3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)